In [1]:
pip install playsound pip install pyttsx3 pyaudio speechrecognition wave

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# Server Code
import socket
import wave
import os
import speech_recognition as sr
import pyttsx3
from IPython.display import Audio, display

HOST = '192.168.0.100'  # Server IP address
PORT = 5000

def process_audio(audio_data):
    with wave.open('audio.wav', 'wb') as f:
        f.setnchannels(1)
        f.setsampwidth(2)
        f.setframerate(44100)  # Sample rate for recorded audio
        f.writeframes(audio_data)

    r = sr.Recognizer()
    with sr.AudioFile('audio.wav') as source:
        audio = r.record(source)
        print("Audio file path:", os.path.abspath('audio.wav'))

    text = ''
    try:
        text = r.recognize_google(audio)
        print("Recognized text:", text)
    except sr.UnknownValueError:
        print("Speech recognition could not understand audio")
    except sr.RequestError as e:
        print("Speech recognition error: {0}".format(e))

    return text, os.path.abspath('audio.wav')

server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server_socket.bind((HOST, PORT))
server_socket.listen(3)

print('Waiting for a connection...')

while True:
    client_socket, address = server_socket.accept()
    print('Connected by', address)

    data = b''
    while True:
        packet = client_socket.recv(1024)
        if b'END_OF_DATA' in packet:
            data += packet.split(b'END_OF_DATA')[0]
            break
        data += packet

    print("Received data length:", len(data))

    audio_data = data.split(b'\nTEXT:')[0].replace(b'AUDIO:', b'')
    user_text = data.split(b'\nTEXT:')[1].decode()

    text, file_path = process_audio(audio_data)
    print("Recognized audio text:", text)
    print("User text:", user_text)

    audio_display = Audio(file_path)
    client_socket.sendall(text.encode() + b'\nEND_OF_TEXT\n')
    client_socket.sendall(audio_data + b'\nEND_OF_AUDIO\n')

    # Convert user text to speech and save it
    engine = pyttsx3.init()
    tts_sample_rate = 22050  # Sample rate for TTS audio
    engine.setProperty('rate', 125)  # Set speech rate to normal (125 words per minute)
    text_audio_file = 'user_text.wav'
    engine.save_to_file(user_text, text_audio_file)
    engine.runAndWait()

    # Send the text audio file to the client
    with open(text_audio_file, 'rb') as f:
        text_audio_data = f.read()

    client_socket.sendall(text_audio_data + b'\nEND_OF_TTS_AUDIO\n')

    # Print binary data for verification
    print("\nBinary data sent to client:")
    print("Text binary:", text.encode())
    print("Audio binary:", audio_data[:20], "...", audio_data[-20:])
    print("TTS audio binary:", text_audio_data[:20], "...", text_audio_data[-20:])

    # Close the client connection
    client_socket.close()
    display(audio_display)
    display(Audio(text_audio_file))

Waiting for a connection...
Connected by ('192.168.0.100', 49279)
Received data length: 880664
Audio file path: C:\Users\varsh\audio.wav
Recognized text: hay hello Namaskar how are you Lala Lala how are you doing I am Varshini Varshini
Recognized audio text: hay hello Namaskar how are you Lala Lala how are you doing I am Varshini Varshini
User text: hii how r u


Binary data sent to client:
Text binary: b'hay hello Namaskar how are you Lala Lala how are you doing I am Varshini Varshini'
Audio binary: b'\x00\x00\x00\x00\xff\xff\x00\x00\x00\x00\x00\x00\x00\x00\xff\xff\x00\x00\x00\x00' ... b'\x8e\x03\xa2\x03\xb1\x03\xb7\x03\xb5\x03\xaa\x03\x9b\x03\x8a\x03u\x03Z\x03'
TTS audio binary: b'RIFFLy\x01\x00WAVEfmt \x12\x00\x00\x00' ... b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'
